# vanilla_VAE:

This is the official implementation of the vanilla_VAE for PyTorch. I just made some minor alterations for the purpose of interfacing.

In [1]:
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
batch_size = 128
epochs = 15
seed = 1
log_interval = 10
z_dim = 5
#database_name = 'mnist'
#database_name = 'cifar10'
database_name = 'fashion'

In [3]:
torch.manual_seed(seed)
device = torch.device("cuda:0")

kwargs = {'num_workers': 1, 'pin_memory': True}
if database_name.lower() == 'mnist':
    database_obj_train = datasets.MNIST('data/mnist', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.MNIST('data/mnist', train=False, transform=transforms.ToTensor())

elif database_name.lower() == 'cifar10':
    database_obj_train = datasets.CIFAR10('data/cifar10', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.CIFAR10('data/cifar10', train=False, transform=transforms.ToTensor())
    
elif database_name.lower() == 'fashion':
    database_obj_train = datasets.FashionMNIST('data/fashion', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.FashionMNIST('data/fashion', train=False, transform=transforms.ToTensor())    
    
    
train_loader = torch.utils.data.DataLoader(database_obj_train,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(database_obj_test,
    batch_size=batch_size, shuffle=True, **kwargs)
try:
    N_train,image_x,image_y,num_channel = train_loader.dataset.data.shape
    data_dim = image_x * image_y * num_channel
except:
    N_train,image_x,image_y = train_loader.dataset.data.shape
    data_dim = image_x * image_y 
    num_channel = 1


100.0%

Extracting data/fashion/FashionMNIST/raw/train-images-idx3-ubyte.gz


20.9%

Extracting data/fashion/FashionMNIST/raw/train-labels-idx1-ubyte.gz


159.1%

Extracting data/fashion/FashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting data/fashion/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(data_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, z_dim)
        self.fc3 = nn.Linear(z_dim, 400)
        self.fc4 = nn.Linear(400, data_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, data_dim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [5]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, data_dim), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


In [8]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, num_channel, image_x, image_y)[:n]])
                save_image(comparison.cpu(),
                         'samples/vanilla_VAE/recon_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))



In [9]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, z_dim).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, num_channel, image_x, image_y),'samples/vanilla_VAE/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 551.199158
Train Epoch: 1 [1280/60000 (2%)]	Loss: 401.530853
Train Epoch: 1 [2560/60000 (4%)]	Loss: 356.759399
Train Epoch: 1 [3840/60000 (6%)]	Loss: 318.495667
Train Epoch: 1 [5120/60000 (9%)]	Loss: 320.254242
Train Epoch: 1 [6400/60000 (11%)]	Loss: 310.338379
Train Epoch: 1 [7680/60000 (13%)]	Loss: 306.762939
Train Epoch: 1 [8960/60000 (15%)]	Loss: 300.857330
Train Epoch: 1 [10240/60000 (17%)]	Loss: 292.307526
Train Epoch: 1 [11520/60000 (19%)]	Loss: 286.517059
Train Epoch: 1 [12800/60000 (21%)]	Loss: 277.143188
Train Epoch: 1 [14080/60000 (23%)]	Loss: 284.326050
Train Epoch: 1 [15360/60000 (26%)]	Loss: 285.218872
Train Epoch: 1 [16640/60000 (28%)]	Loss: 274.313965
Train Epoch: 1 [17920/60000 (30%)]	Loss: 261.376282
Train Epoch: 1 [19200/60000 (32%)]	Loss: 268.726624
Train Epoch: 1 [20480/60000 (34%)]	Loss: 265.785950
Train Epoch: 1 [21760/60000 (36%)]	Loss: 279.026978
Train Epoch: 1 [23040/60000 (38%)]	Loss: 270.238708
Train Epoch: 1 [24320/60000 